In [1]:
import pybullet as p
import time
import cv2
import numpy as np

import numpy as np
# define a class for the agent
class WanderBot:
    def __init__(self,max_ray_dist, max_ray_angle, n_ray_per_side, ray_sensor_height,
                 default_speed, no_obs_tau, min_no_obs_t, update_dt, left_tau, right_tau,
                cam_width=640, cam_height=480, center_thresh=0.05,reward_scale=1000):
        self.max_ray_dist = max_ray_dist
        self.max_ray_angle = max_ray_angle
        self.n_ray_per_side = n_ray_per_side
        if n_ray_per_side==1:
            self.ray_angle_delta = max_ray_angle
        else:
            self.ray_angle_delta = max_ray_angle / (n_ray_per_side-0.5)
        self.ray_sensor_height = ray_sensor_height
        self.default_speed = default_speed
        self.no_obs_cumu_t = 0.0 
        self.no_obs_tau = no_obs_tau
        self.min_no_obs_t = min_no_obs_t
        self.no_obs_state = 0
        self.update_dt = update_dt
        self.left_tau = left_tau
        self.left_cumu_t = 0.
        self.right_tau = right_tau
        self.right_cumu_t = 0.
        self.cam_w = cam_width
        self.cam_h = cam_height
        self.center_thresh = center_thresh
        self.reward_scale = reward_scale
    
    def get_current_position_and_orientation(self,p,agent):
        position, quaternion = p.getBasePositionAndOrientation(agent)
        x = position[0]
        y = position[1]
        eular_angle = p.getEulerFromQuaternion(quaternion)
        yaw = eular_angle[2] # only need yaw
        return x,y,yaw
    
    def dist_to_target(self,p,agent,cpos):
        x,y,_ = self.get_current_position_and_orientation(p,agent)
        d = np.power(x-cpos[0],2)+np.power(y-cpos[1],2)
        d = np.sqrt(d)
        return d
    
    def get_image(self,p,agent):
        x,y,yaw = self.get_current_position_and_orientation(p,agent)
        h = 0.2
        d1 = 0.2
        cam_pos = [x+d1*np.cos(yaw),y+d1*np.sin(yaw),h]
        d = 1.
        cam_target_pos = [x+d*np.cos(yaw),y+d*np.sin(yaw),h]
        cam_up_vec = [0.,0.,1.0]
        view_matrix = p.computeViewMatrix(cam_pos,cam_target_pos,cam_up_vec)
        projection_matrix = p.computeProjectionMatrixFOV(
                            fov=90, aspect=4./3., nearVal=0.1, farVal=10.0)
        width = 640
        height = 480
        data = p.getCameraImage(width,
                          height,
                          view_matrix,
                          projection_matrix,
                          shadow=True,
                          renderer=p.ER_BULLET_HARDWARE_OPENGL)
        
        return data[2]
    
    def get_object_size_center(self,image):
        im_hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
        mask = cv2.inRange(im_hsv,(40,0,0),(60,255,255))
        size = np.sum(mask==255)
        if size>0:
            M = cv2.moments(mask)
            # calculate x,y coordinate of center
            cX = M["m10"] / M["m00"]
            cY = M["m01"] / M["m00"]
            cX = cX/self.cam_w
            cY = cY/self.cam_h
            #print(cX,cY,size)
            size_re = size / self.cam_w / self.cam_h
            return (cX,cY),size_re
        else:
            return (-1,-1),0.0
    
    def compute_reward(self,center,size):
        x = center[0]
        x0 = (x-self.cam_w/2)/self.cam_w
        if np.abs(x0) < self.center_thresh/2:
            r = self.reward_scale*size/self.cam_w/self.cam_h
        else:
            r = 0
        return r
        
    # send ray for testing obstacle
    def ray_test(self,side,p,agent):
        # first figure out the position and orientation of the robot
        x,y,yaw = self.get_current_position_and_orientation(p,agent)
        # print(x,y,yaw)
        # construct the list of rays to send
        rayFrom = []
        rayTo = []
        init_angle = self.ray_angle_delta/2
        delta_angle = self.ray_angle_delta
        if side=="right":
            init_angle = -init_angle
            delta_angle = -delta_angle
        
        for i in range(self.n_ray_per_side):
            rayFrom.append([x,y,self.ray_sensor_height])
            rayAngle = yaw + init_angle + i*delta_angle
            rayTo.append([x+self.max_ray_dist*np.cos(rayAngle),
                          y+self.max_ray_dist*np.sin(rayAngle),
                          self.ray_sensor_height])
        #print(rayTo)
        # send array
        results = p.rayTestBatch(rayFrom,rayTo)
        hit_object_id = []
        n_ray_hit = 0
        for result in results:
            hit_object_id.append(result[0])
            if result[0]!=-1:
                # count how many rays hit an object
                # this number is used to drive motor
                n_ray_hit += 1 
    
        return n_ray_hit,hit_object_id,rayTo
    
    def set_motor_velocity(self,p,agent,left_vel,right_vel):
        p.setJointMotorControl2(agent,0,p.VELOCITY_CONTROL,
                                targetVelocity=left_vel*self.default_speed,force=1000)
        p.setJointMotorControl2(agent,1,p.VELOCITY_CONTROL,
                                targetVelocity=right_vel*self.default_speed,force=1000)
        
    def check_no_obs_state(self):
        # determine whether the robot enters the no-obstacle
        # state, which prob increases linearly after the min
        # time threshold
        ct = self.no_obs_cumu_t - self.min_no_obs_t
        ct = max(0.,ct)
        prob = ct / self.no_obs_tau
        prob = min(1.,prob)
        if np.random.rand() < prob and self.no_obs_state == 0:
            # print("entered no obs state")
            self.no_obs_state = 1
            self.left_cumu_t = 0.0
            self.right_cumu_t = 0.0
            return 1
        else:
            return 0
    
    def get_motor_velocity(self,n_ray_left,n_ray_right,left_vel,right_vel):
        if n_ray_right > 0 or n_ray_left > 0:
            # there is obstacle, reset no-obstacle state
            self.no_obs_cumu_t = 0.0
            self.no_obs_state = 0
        else:
            # otherwise increase time counter
            self.no_obs_cumu_t += self.update_dt
            # check whether entered no-obstacle state
            self.check_no_obs_state()
        
        if self.no_obs_state == 0 : # there is obstacle
            dv_obs = 0.5
            if n_ray_right > 0:
                left_vel -= dv_obs*(np.random.rand())
            else:
                left_vel += dv_obs*(np.random.rand())
            if n_ray_left > 0:
                right_vel -= dv_obs*(np.random.rand())
            else:
                right_vel += dv_obs*(np.random.rand())
        
        else: # in no obstacle state, vary left and right wheel 
              # speed according to other rule
                
            # increase time counter
            self.left_cumu_t += self.update_dt
            self.right_cumu_t += self.update_dt
            
            # vary motor speed according to time in no-obstacle state
            left_vel = self.vary_motor_vel("left",left_vel)
            right_vel = self.vary_motor_vel("right",right_vel)
            
            # by default, slowly increase speed of both wheels
            dv_no_obs = 0.05
            left_vel += dv_no_obs*(np.random.rand())
            right_vel += dv_no_obs*(np.random.rand())
        
        left_vel = min(left_vel, 1.0)
        left_vel = max(left_vel,-1.0)
        
        right_vel = min(right_vel, 1.0)
        right_vel = max(right_vel,-1.0)
           
        return left_vel, right_vel
        
    def vary_motor_vel(self,side,vel):
        if side == "left":
            prob = self.left_cumu_t / self.left_tau
        else:
            prob = self.right_cumu_t / self.right_tau
        prob = min(1.0,prob)
        tmp = np.random.rand()
        if tmp < prob:
            # reset time counter
            if side == "left":
                # print("left wheel speed varied",tmp,prob,self.left_cumu_t)
                self.left_cumu_t = 0.0
            else:
                # print("right wheel speed varied",tmp,prob,self.right_cumu_t)
                self.right_cumu_t = 0.0
            
            # randomly choose a vel between 0 and 1
            return np.random.rand()
        else:
            # otherwise return the original velocity
            return vel

class RL_agent():
    def __init__(self,n_wheel_state,hori_state_edge,size_state_edge,epsilon,
                 alpha,gamma,reward_bin):
        n_action_per_wheel = 3 # either increase/no change/decrease
        self.wheel_state_list = np.linspace(-1.0,1.0,n_wheel_state,endpoint=True)
        self.vel_step = self.wheel_state_list[1] - self.wheel_state_list[0]
        self.hori_state_edge = hori_state_edge
        self.size_state_edge = size_state_edge
        n_hori_state = len(hori_state_edge)-1
        n_size_state_edge = len(size_state_edge)-1
        self.Q = np.zeros((n_hori_state,n_size_state_edge,n_wheel_state,n_wheel_state,
                           n_action_per_wheel,n_action_per_wheel))
        # the state: 1. horizontal location of the target object (0 to 1)
        #            2. left wheel velocity (-1 to 1)
        #            3. right wheel velocity (-1 to 1)
        self.h = 0.
        self.s = 0.
        self.left_w_vel = 0.
        self.right_w_vel = 0.
        self.action_list = [-self.vel_step,0.,self.vel_step]
        self.naction = len(self.action_list)
        self.At = np.ones((3,2),dtype=np.int16) # default 0 velocity
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.reward_bin = reward_bin
        
    def get_hori_state_ind(self,h):
        for i in range(len(self.hori_state_edge)-1):
            if h>=self.hori_state_edge[i] and h<self.hori_state_edge[i+1]:
                return i
        i = len(self.hori_state_edge)-2
        if h>=self.hori_state_edge[i] and h<=self.hori_state_edge[i+1]:
            return i
    
    def get_size_state_ind(self,s):
        for i in range(len(self.size_state_edge)-1):
            if s>=self.size_state_edge[i] and s<self.size_state_edge[i+1]:
                return i
        i = len(self.size_state_edge)-2
        if s>=self.size_state_edge[i] and s<=self.size_state_edge[i+1]:
            return i
    
    def get_wheel_state_ind(self,w_vel):
        ind = int((w_vel - (-1.0))/self.vel_step)
        return ind
    
    def get_state_ind(self):
        iH = self.get_hori_state_ind(self.h)
        iS = self.get_size_state_ind(self.s)
        iWL = self.get_wheel_state_ind(self.left_w_vel)
        iWR = self.get_wheel_state_ind(self.right_w_vel)
        return [iH,iS,iWL,iWR]
    
    def select_action(self,h):
        p = np.random.rand()
        if p<self.epsilon: # select the action with largest Q
            iH,iS,iWL,iWR = self.get_state_ind()
            Q_this = self.Q[iH,iS,iWL,iWR,:,:]
            # find max
            Q_max = np.max(Q_this.flatten())
            imax = np.argwhere(Q_this==Q_max)
            i0 = np.random.permutation(imax.shape[0])[0]
            iL = imax[i0,0]
            iR = imax[i0,1]
        else:
            # random select
            iL = np.random.permutation(self.naction)[0]
            iR = np.random.permutation(self.naction)[0]
        dVL = self.action_list[iL]
        dVR = self.action_list[iR]
        return [iL,iR],dVL,dVR
    
    def update_Q(self,last_state,last_action,curr_state,reward):
        Q_last = self.Q[last_state[0],last_state[1],last_state[2],last_state[3],
                        last_action[0],last_action[1]]
        Q_max = np.max(self.Q[curr_state[0],curr_state[1],curr_state[2],curr_state[3],:,:])
        self.Q[last_state[0],last_state[1],last_state[2],last_state[3],
                        last_action[0],last_action[1]] = Q_last + self.alpha*(reward+self.gamma*Q_max-Q_last)
    
    def update_vel(self,dVL,dVR):
        self.left_w_vel += dVL
        self.right_w_vel += dVR
        self.left_w_vel = min(self.left_w_vel,1.0)
        self.left_w_vel = max(self.left_w_vel,-1.0)
        self.right_w_vel = min(self.right_w_vel,1.0)
        self.right_w_vel = max(self.right_w_vel,-1.0)
    
    def set_hs(self,h,s):
        self.h = h
        self.s = s
        
    def compute_reward(self,size):
        iH = self.get_hori_state_ind(self.h)
        if iH == self.reward_bin:
            r = size
        else:
            r = -1
        return r
    
    def save(self):
        import pickle # for saving data
        # create time stamp
        from datetime import datetime
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
        # create file name
        fn = "RL_agent_"+date_time+".data"
        # save data
        with open(fn, 'wb') as file2save:
            pickle.dump(self, file2save)
        print("Data saved to ",fn)
    
    def load(self,fn):
        import pickle
        with open(fn, 'rb') as file:
            return pickle.load(file)



In [5]:
use_gui = True

try:
    p.disconnect()
except:
    print("reconnecting...")
    
if use_gui:
    p.connect(p.GUI)
    p.setRealTimeSimulation(1)
    # the default delta t for pybullet is 1/240 sec
    sim_dt = 1./240.
    p.setTimeStep(sim_dt)
    p.resetDebugVisualizerCamera( cameraDistance=4, cameraYaw=0,
                    cameraPitch=-45, cameraTargetPosition=[1.0,0,0])
else:
    p.connect(p.DIRECT)
    p.setRealTimeSimulation(0)
    sim_dt = 1./100.
    p.setTimeStep(sim_dt)
    
p.setGravity(0,0,-9.8)

# spawn ground plane
plane = p.loadURDF("plane.urdf")


# spawn the cylindral obstacles
cylinder_pos_array = [[2.0,-2.0]]
cylinder_list = []
for i,cpos in enumerate(cylinder_pos_array):
    cylinder = p.loadURDF("cylinder.urdf",basePosition=[cpos[0],cpos[1],0.05],useFixedBase=1)
    cylinder_list.append(cylinder)
    
# spawn turtle bot
init_pos = [0.,0.,0.]
init_orientation = [0.,0.,0.,1.]
turtle = p.loadURDF("turtlebot.urdf",init_pos,init_orientation)
time.sleep(5.0)
# specify the delta t for the agent update its, which is different
# from simulation time step
agent_update_dt = 0.020
n_update = int(np.round(agent_update_dt / sim_dt))

wb = WanderBot(max_ray_dist=0.4, max_ray_angle=60.0/180.0*np.pi, n_ray_per_side=7, 
               ray_sensor_height=0.45,default_speed=5.,no_obs_tau=1.0, min_no_obs_t=0.5, 
               update_dt = agent_update_dt, left_tau=2.0,right_tau=2.0,
               cam_width=640, cam_height=480, center_thresh=0.08,reward_scale=1000)

size_edge = np.logspace(-4,np.log10(0.5),8,endpoint=True)
size_edge = np.hstack((0.0,size_edge))

RL = RL_agent(n_wheel_state=5,
                hori_state_edge=[-1.0,0.0,0.4,0.6,1.0],
                size_state_edge=size_edge,
                epsilon=0.5,alpha=0.5,gamma=0.9,reward_bin=2)

use_prev = 1
test_mode = 1
if use_prev==1:
    RL = RL.load('RL_agent_2023-06-09_19-29-42.data')

    if test_mode:
        RL.epsilon = 1.0
        RL.alpha = 0.0
    else:
        RL.epsilon = 0.9
        RL.alpha = 0.5
    # reset wheel velocity
    RL.left_w_vel = 0.
    RL.right_w_vel = 0.


# maximum duration for each episode in seconds
max_dur_per_episode = 30.0
if test_mode:
    max_episode = 1
else:
    max_episode = 50
i_episode = 0
stop_all=False
while stop_all == False:
    # reset model
    p.resetBasePositionAndOrientation(turtle,init_pos,init_orientation)
    
    # start the current episode
    stop = False
    t_lapsed = 0.
    i = 0

    # start video recording 
    from datetime import datetime
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d_%H-%M-%S")
    vidLogger = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, "Turtlebot_RL_Controller_"+date_time+".mp4")

    # intialize state
    image = wb.get_image(p,turtle)
    center,size = wb.get_object_size_center(image)
    h = center[0]
    RL.set_hs(h,size)
    last_state = RL.get_state_ind()
    last_action = [1,1] # 1 is dV = 0

    turtle_pos = []

    time.sleep(1.0)
    if i_episode>=1 and i_episode%5==0:
        RL.alpha = RL.alpha*0.9
    print('-------- episode ',i_episode+1,'---------')
    while stop==False:
        if use_gui:
            time.sleep(1./240.)
            keys = p.getKeyboardEvents()
            for k,v in keys.items():
                    if (k == ord('q') and (v&p.KEY_WAS_TRIGGERED)):
                        stop=True
        else:
            p.stepSimulation()

        if i % n_update == 0:
            x,y,yaw = wb.get_current_position_and_orientation(p,turtle)
            
            turtle_pos.append([x,y])
            # get current state
            image = wb.get_image(p,turtle)
            center,size = wb.get_object_size_center(image)
            h = center[0]
            RL.set_hs(h,size)
            curr_state = RL.get_state_ind()

            # get reward for last action
            reward = RL.compute_reward(size*1e3) 

            # select next action based on current state
            curr_action,dVL,dVR = RL.select_action(center[0])

            # use reward to update last state
            RL.update_Q(last_state,last_action,curr_state,reward)

            # cache last state and action
            last_state = curr_state
            last_action = curr_action

            # carry out the action
            RL.update_vel(dVL, dVR)
        # command wheel speed
        wb.set_motor_velocity(p,turtle,RL.left_w_vel,RL.right_w_vel)

        d = wb.dist_to_target(p,turtle,cylinder_pos_array[0])
        if d>=5 or d<=0.7:
            stop = True

        if i % int(200) == 0:
            # print(center,size,reward)
            p_update = sum(RL.Q.flatten()!=0)/RL.Q.size*100
            print('curr state',curr_state,'curr action',curr_action,'reward',reward,'distance',d,'percent',p_update)
        i+=1

        t_lapsed += sim_dt
        if t_lapsed>=max_dur_per_episode:
            stop = True
        if stop:
            break
    i_episode+=1
    turtle_pos = np.array(turtle_pos)
    if p_update>=50 or i_episode>=max_episode:
        stop_all=True

print('done')
wb.set_motor_velocity(p,turtle,0,0)
time.sleep(5.0)
p.stopStateLogging(vidLogger)
p.disconnect()

RL.save()

reconnecting...
-------- episode  1 ---------
curr state [3, 3, 2, 2] curr action [2, 0] reward -1 distance 2.828457616708146 percent 50.47222222222222
curr state [3, 3, 3, 4] curr action [1, 1] reward -1 distance 2.3890773486192756 percent 50.47222222222222
curr state [3, 4, 3, 4] curr action [2, 0] reward -1 distance 2.073919440232415 percent 50.47222222222222
curr state [3, 3, 3, 3] curr action [1, 2] reward -1 distance 1.7471300833511991 percent 50.47222222222222
curr state [2, 4, 4, 4] curr action [1, 1] reward 5.771484375 distance 1.296014571880692 percent 50.47222222222222
curr state [2, 5, 3, 3] curr action [0, 0] reward 16.259765625 distance 0.853512372032726 percent 50.47222222222222
curr state [1, 5, 2, 2] curr action [0, 2] reward -1 distance 0.7338705021656022 percent 50.47222222222222
done
Data saved to  RL_agent_2023-06-09_20-18-38.data
